Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f, encoding = 'latin1')
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

# L2 regularization for Logistic model

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 5e-3

graph = tf.Graph()
with graph.as_default():

# Input data
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

# Variables
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases  = tf.Variable(tf.zeros([num_labels]))


  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  regularization = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regularization)

# Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Prediction
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 1001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 36.029976
Training accuracy: 8.6%
Validation accuracy: 9.5%
Loss at step 100: 10.525558
Training accuracy: 72.0%
Validation accuracy: 70.5%
Loss at step 200: 6.219536
Training accuracy: 76.7%
Validation accuracy: 74.1%
Loss at step 300: 3.811453
Training accuracy: 79.4%
Validation accuracy: 76.2%
Loss at step 400: 2.447153
Training accuracy: 81.8%
Validation accuracy: 78.2%
Loss at step 500: 1.672443
Training accuracy: 83.4%
Validation accuracy: 79.6%
Loss at step 600: 1.230562
Training accuracy: 84.4%
Validation accuracy: 80.6%
Loss at step 700: 0.976998
Training accuracy: 85.0%
Validation accuracy: 81.1%
Loss at step 800: 0.830612
Training accuracy: 85.4%
Validation accuracy: 81.4%
Loss at step 900: 0.745638
Training accuracy: 85.6%
Validation accuracy: 81.6%
Loss at step 1000: 0.696081
Training accuracy: 85.7%
Validation accuracy: 81.8%
Test accuracy: 88.8%


# L2 regularization for Neural Network model

In [7]:
# parameters
batch_size = 128
image_size = 28

# network params
num_labels = 10 
num_hidden = 1024 
beta = 5e-3


graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ## Build a hidden layer with 1024 neurons.
    # Variables. (hidden)
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    # output layer
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 ## 
    # ReLUs
    relu_layer = tf.nn.relu(logits_1)
    
    logits = tf.matmul(relu_layer, weights_2) + biases_2
    
    # loss function + L2 regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) 
    
    # L2 regularization
    regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta*regularization)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_valid = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_valid)
    
    # Predictions for test data
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_test = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_test)

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
#   tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1939.893677
Minibatch accuracy: 12.5%
Validation accuracy: 31.5%
Minibatch loss at step 500: 127.826668
Minibatch accuracy: 82.8%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 11.010814
Minibatch accuracy: 83.6%
Validation accuracy: 84.8%
Test accuracy: 91.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 1001

# Restrict the training data to just a few batches
train_dataset_few = train_dataset[:500,:]
train_labels_few = train_labels[:500]

with tf.Session(graph=graph) as session:
#   tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_few.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_few[offset:(offset + batch_size), :]
    batch_labels = train_labels_few[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1946.343262
Minibatch accuracy: 8.6%
Validation accuracy: 31.7%
Minibatch loss at step 500: 128.414581
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 10.588015
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Test accuracy: 85.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
# parameters
batch_size = 128
image_size = 28

# network params
num_labels = 10 
num_hidden = 1024 
beta = 0.01

# create tensorflow graph
graph = tf.Graph()
with graph.as_default():
    # tensorflow graph input
    # At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ## Build a hidden layer with 1024 neurons.
    # Variables. (hidden)
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    # output layer
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 ## 
    # ReLUs
    relu_layer = tf.nn.relu(logits_1)
    
    # dropout 
   
    keep_prob = tf.placeholder(tf.float32) 
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    
    # output layer
    logits = tf.matmul(relu_layer_dropout, weights_2) + biases_2
    
    # loss function + L2 regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) 
    
    # L2 regularization
    regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta*regularization)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_valid = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_valid)
    
    # Predictions for test data
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_test = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_test)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3567.369385
Minibatch accuracy: 10.9%
Validation accuracy: 27.6%
Minibatch loss at step 500: 21.300634
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.996845
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.638317
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.724364
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.778409
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.833780
Minibatch accuracy: 80.5%
Validation accuracy: 83.2%
Test accuracy: 89.9%


# Question: What happens to our extreme overfitting case?

In [14]:
num_steps = 3001

train_dataset_few = train_dataset[:500,:]
train_labels_few = train_labels[:500]

with tf.Session(graph=graph) as session:
#   tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_few.shape[0] - batch_size)
    batch_data = train_dataset_few[offset:(offset + batch_size), :]
    batch_labels = train_labels_few[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3664.218750
Minibatch accuracy: 9.4%
Validation accuracy: 29.6%
Minibatch loss at step 500: 21.118073
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 0.509262
Minibatch accuracy: 99.2%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 0.325180
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.317957
Minibatch accuracy: 99.2%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 0.309969
Minibatch accuracy: 99.2%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 0.298528
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Test accuracy: 86.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [18]:
import math as math

batch_size = 128
image_size = 28
num_labels = 10 

beta = 0.05
num_hidden_neurons_1 = 1024
num_hidden_neurons_2 = int(num_hidden_neurons_1 * np.power(0.5, 1)) 
num_hidden_neurons_3 = int(num_hidden_neurons_1 * np.power(0.5, 2))
num_hidden_neurons_4 = int(num_hidden_neurons_1 * np.power(0.5, 3)) 
num_hidden_neurons_5 = int(num_hidden_neurons_1 * np.power(0.5, 4)) 

# create tensorflow graph
graph = tf.Graph()
with graph.as_default():
    # tensorflow graph input
    # At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ## Build four(4) hidden layers with 1024 neurons at each layer.
    ## Hidden layer 1 with ReLUs
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_neurons_1], stddev=math.sqrt(2.0/(image_size * image_size)))) ## stddev normal distribution
    biases_1 = tf.Variable(tf.zeros([num_hidden_neurons_1]))
    
    ## Hidden layer 2 with ReLUs
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_neurons_1, num_hidden_neurons_2], stddev=math.sqrt(2.0/(num_hidden_neurons_1)))) 
    biases_2 = tf.Variable(tf.zeros([num_hidden_neurons_2]))
    
    ## Hidden layer 3 with ReLUs
    weights_3 = tf.Variable(tf.truncated_normal([num_hidden_neurons_2, num_hidden_neurons_3], stddev=math.sqrt(2.0/(num_hidden_neurons_2)))) 
    biases_3 = tf.Variable(tf.zeros([num_hidden_neurons_3]))
    
    ## Hidden layer 4 with ReLUs
    weights_4 = tf.Variable(tf.truncated_normal([num_hidden_neurons_3, num_hidden_neurons_4], stddev=math.sqrt(2.0/(num_hidden_neurons_3)))) 
    biases_4 = tf.Variable(tf.zeros([num_hidden_neurons_4]))
    
    ## Hidden layer 5 with ReLUs
    weights_5 = tf.Variable(tf.truncated_normal([num_hidden_neurons_4, num_hidden_neurons_5], stddev=math.sqrt(2.0/(num_hidden_neurons_4)))) 
    biases_5 = tf.Variable(tf.zeros([num_hidden_neurons_5]))
    
        
    # output layer
    weights_6 = tf.Variable(tf.truncated_normal([num_hidden_neurons_5, num_labels],stddev=math.sqrt(2.0/(num_hidden_neurons_5))))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
                            
    # Training computation.
    # Hidden layer 1 with ReLUs
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 
    relu_layer_1 = tf.nn.relu(logits_1)
    # dropout 
    keep_prob = tf.placeholder(tf.float32) 
    relu_layer_dropout_1 = tf.nn.dropout(relu_layer_1, keep_prob)
    
    ## Hidden layer 2 with ReLUs
    logits_2 = tf.matmul(relu_layer_dropout_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    relu_layer_dropout_2 = tf.nn.dropout(relu_layer_2, keep_prob)
    
    ## Hidden layer 3 with ReLUs
    logits_3 = tf.matmul(relu_layer_dropout_2, weights_3) + biases_3
    relu_layer_3 = tf.nn.relu(logits_3)
    relu_layer_dropout_3 = tf.nn.dropout(relu_layer_3, keep_prob)
    
    ## Hidden layer 4 with ReLUs
    logits_4 = tf.matmul(relu_layer_dropout_3, weights_4) + biases_4
    relu_layer_4 = tf.nn.relu(logits_4)
    relu_layer_dropout_4 = tf.nn.dropout(relu_layer_4, keep_prob)
                            
    ## Hidden layer 5 with ReLUs
    logits_5 = tf.matmul(relu_layer_dropout_4, weights_5) + biases_5
    relu_layer_5 = tf.nn.relu(logits_5)
    relu_layer_dropout_5 = tf.nn.dropout(relu_layer_5, keep_prob)
    
    # output layer
    logits = tf.matmul(relu_layer_dropout_5, weights_6) + biases_6
    
    # loss function + L2 regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) 
    
    # L2 regularization
    regularization = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta*regularization)
    
    # Optimizer.
    ## Use Learning Rate Decaying.
    global_step = tf.Variable(0) ## count the number of steps taken
    starter_learning_rate = 0.5
    number_decay_steps = 100000
    base_of = 0.96

    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, number_decay_steps, base_of, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for validation 
    logits_valid_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_valid_1 = tf.nn.relu(logits_valid_1)
    # l2
    logits_valid_2 = tf.matmul(relu_layer_valid_1, weights_2) + biases_2
    relu_layer_valid_2 = tf.nn.relu(logits_valid_2)
    # l3
    logits_valid_3 = tf.matmul(relu_layer_valid_2, weights_3) + biases_3
    relu_layer_valid_3 = tf.nn.relu(logits_valid_3)
    # l4
    logits_valid_4 = tf.matmul(relu_layer_valid_3, weights_4) + biases_4
    relu_layer_valid_4 = tf.nn.relu(logits_valid_4)
    # l5
    logits_valid_5 = tf.matmul(relu_layer_valid_4, weights_5) + biases_5
    relu_layer_valid_5 = tf.nn.relu(logits_valid_5)
    
    logits_valid = tf.matmul(relu_layer_valid_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(logits_valid)
    
    # Predictions for test data
    logits_test_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_test_1 = tf.nn.relu(logits_test_1)
    # l2
    logits_test_2 = tf.matmul(relu_layer_test_1, weights_2) + biases_2
    relu_layer_test_2 = tf.nn.relu(logits_test_2)
    # l3
    logits_test_3 = tf.matmul(relu_layer_test_2, weights_3) + biases_3
    relu_layer_test_3 = tf.nn.relu(logits_test_3)
    # l4
    logits_test_4 = tf.matmul(relu_layer_test_3, weights_4) + biases_4
    relu_layer_test_4 = tf.nn.relu(logits_test_4)
    # l5
    logits_test_5 = tf.matmul(relu_layer_test_4, weights_5) + biases_5
    relu_layer_test_5 = tf.nn.relu(logits_test_5)
                                     
    logits_test = tf.matmul(relu_layer_test_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(logits_test)

In [16]:
num_steps = 10001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.195339
Minibatch accuracy: 8.6%
Validation accuracy: 6.2%
Minibatch loss at step 500: 1.758227
Minibatch accuracy: 83.6%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 1.471026
Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 1.102370
Minibatch accuracy: 83.6%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 0.884799
Minibatch accuracy: 90.6%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 0.927663
Minibatch accuracy: 82.8%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 0.810785
Minibatch accuracy: 88.3%
Validation accuracy: 84.9%
Minibatch loss at step 3500: 0.865958
Minibatch accuracy: 82.8%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 0.846156
Minibatch accuracy: 85.9%
Validation accuracy: 85.4%
Minibatch loss at step 4500: 0.802415
Minibatch accuracy: 82.0%
Validation accuracy: 85.5%
Minibatch loss at step 5000: 0.800845
Minibatch accuracy: 82.8%
Validation accuracy: